<a href="https://colab.research.google.com/github/avdlaan/Angular-10-Templates/blob/master/Ref_Bert_3_06_Text_classification_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification 

Let's learn how to finetune the pre-trained BERT for text classification tasks. Say, we are performing sentiment analysis. In the sentiment analysis, our goal is to classify whether a sentence is positive or negative. Suppose, we have a dataset containing sentences along with their labels. 

Consider a sentence: 'I love Pairs'. First, we tokenize the sentence, add the [CLS] token at the beginning, and [SEP] token at the end of the sentence. Then, we feed the tokens as an input to the pre-trained BERT and get the embeddings of all the tokens. 

Next, we ignore the embedding of all other tokens and take only the embedding of [CLS] token which is $R_{[CLS]}$. The embedding of the [CLS] token will hold the aggregate representation of the sentence. We feed $R_{[CLS]}$ to a classifier (feed-forward network with softmax function) and train the classifier to perform sentiment analysis. 

Wait! How does it differ from what we saw at the beginning of the section. How finetuning the pre-trained BERT differs from using the pre-trained BERT as a feature extractor?

In "Extracting embeddings from pre-trained BERT" section, we learned that after extracting the embedding $R_{[CLS]}$ of a sentence, we feed the $R_{[CLS]}$ to a classifier and train the classifier to perform classification. Similarly, during finetuning, we feed the embedding of $R_{[CLS]}$ to a classifier and train the classifier to perform classification.

The difference is that when we finetune the pre-trained BERT, we can update the weights of the pre-trained BERT along with a classifier. But when we use the pre-trained BERT as a feature extractor, we can update only the weights of a classifier and not the pre-trained BERT. 

During finetuning, we can adjust the weights of the model in the following two ways:

- Update the weights of the pre-trained BERT along with the classification layer 
- Update only the weights of the classification layer and not the pre-trained BERT. When we do this, it becomes the same as using the pre-trained BERT as a feature extractor

The following figure shows how we finetune the pre-trained BERT for the sentiment analysis task:


![title](images/6.png)

As we can observe from the preceding figure, we feed the tokens to the pre-trained BERT and get the embedding of all the tokens. We take the embedding of [CLS] token and feed it to a feedforward network with a softmax function and perform classification. 

Let's get a better understanding of how finetuning works by getting hands-on with finetuning the pre-trained BERT for sentiment analysis in the next section. 

# Finetuning BERT for sentiment analysis 
Let's explore how to finetune the pre-trained BERT for a sentiment analysis task with the IMDB dataset. The IMDB dataset consists of movie reviews along with the respective sentiment. The dataset used in this section can be downloaded from here. [link TBA] 

Import the dependencies 

First, let's install the necessary libraries: 

In [1]:
!pip install nlp
!pip install transformers==3.5.1
!pip install -U torchtext==0.8.0
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install datasets



Import the necessary modules: 

In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
import nlp
import torch
import numpy as np


Load the model  and dataset. Let's load the dataset using the nlp library:  

In [3]:
from datasets import load_dataset
dataset = load_dataset('imdb')

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)



Let us check the datatype:

In [4]:
type(dataset)

datasets.dataset_dict.DatasetDict


Next, let's split the dataset into train and test set:

In [5]:
dataset = (load_dataset('imdb', split='train')
        .train_test_split(test_size=0.3))


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-5aad2f09802ed83c.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-f974c523b8ee0ed3.arrow



Let's print the dataset:

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 17500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7500
    })
})


Now, we create the  train and test sets:




In [7]:
train_set = dataset["train"]
test_set = dataset["test"]


Next, let's download and load the pre-trained BERT model. In this example, we use the pre-trained bert-base-uncased model. As we can observe below, since we are performing sequence classification, we use the BertForSequenceClassification class: 


In [8]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


Next, we download and load the tokenizer which is used for pretraining the bert-base-uncased model.
As we can observe, we create the tokenizer using the BertTokenizerFastclass instead of BertTokenizer. The BertTokenizerFast class has many advantages compared to BertTokenizer. We will learn about this in the next section: 


In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


Now that we loaded the dataset and model, next let's preprocess the dataset. 

## Preprocess the dataset
We can preprocess the dataset in a quicker way using our tokenizer. For example, consider the sentence: 'I love Paris'.  

First, we tokenize the sentence and add the [CLS] token at the beginning and [SEP] token at the end as shown below: 


In [10]:
tokens = [ '[CLS]', 'I', 'love', 'Paris', '[SEP]' ]


Next, we map the tokens to the unique input ids (token ids). Suppose the following are the unique input ids (token ids):


In [11]:
input_ids = [101, 1045, 2293, 3000, 102]

Then, we need to add the segment ids (token type ids). Wait, what are segment ids? Suppose we have two sentences in the input. In that case, segment ids are used to distinguish one sentence from the other. All the tokens from the first sentence will be mapped to 0 and all the tokens from the second sentence will be mapped to 1. Since here we have only one sentence, all the tokens will be mapped to 0 as shown below:


In [12]:

token_type_ids = [0, 0, 0, 0, 0]


Now, we need to create the attention mask. We know that an attention mask is used to differentiate the actual tokens and [PAD] tokens. It will map all the actual tokens to 1 and the [PAD] tokens to 0. Suppose, our tokens length should be 5. Now, our tokens list has already 5 tokens. So, we don't have to add [PAD] token. Then our attention mask will become: 


In [13]:
attention_mask = [1, 1, 1, 1, 1]


That's it. But instead of doing all the above steps manually, our tokenizer will do these steps for us. We just need to pass the sentence to the tokenizer as shown below: 


In [14]:
tokenizer('I love Paris')

{'input_ids': [101, 1045, 2293, 3000, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}


With the tokenizer, we can also pass any number of sentences and perform padding dynamically. To do that, we need to set padding to True and also the maximum sequence length. For instance, as shown below, we pass three sentences and we set the maximum sequence length, max_length to 5:


In [15]:
tokenizer(['I love Paris','birds fly','snow fall'], padding = True, max_length=5,truncation=True,add_special_tokens = True,return_length=True)

{'input_ids': [[101, 1045, 2293, 3000, 102], [101, 5055, 4875, 102, 0], [101, 4586, 2991, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 0]], 'length': [5, 5, 5]}


That's it, with the tokenizer, we can easily preprocess our dataset. So we define a function called preprocess for processing the dataset as shown below: 


In [16]:

def preprocess(data):
    return tokenizer(data['text'], padding=True, truncation=True,max_length=5,add_special_tokens = True)

    


Now, we preprocess the train and test set using the preprocess function: 


In [17]:
train_set = train_set.map(preprocess, batched=True, batch_size=len(train_set)) 
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]


Next, we use the set_format function and select the columns which we need in our dataset and also in which format we need them as shown below:  


In [18]:
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

That's it. Now that we have the dataset ready, let's train the model. 

## Training the model 


Define the batch size and epoch size: 

In [19]:
batch_size = 8
epochs = 2


Define the warmup steps and weight decay:

In [20]:
warmup_steps = 500
weight_decay = 0.01


Define the training arguments:

In [21]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    evaluate_during_training=True,
    logging_dir='./logs',
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,




Now define the trainer: 

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)



Start training the model:

In [23]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.699183,0.711165
1000,0.696271,0.693090
1500,0.699318,0.693219
2000,0.697569,0.693102
2500,0.695290,0.694583
3000,0.695445,0.693117
3500,0.694804,0.693865
4000,0.694401,0.693297


TrainOutput(global_step=4376, training_loss=0.6962501762989888)


After training we can evaluate the model using the evaluate function:

In [24]:
trainer.evaluate()

{'epoch': 2.0, 'eval_loss': 0.6932874917984009}

In [26]:
trainer.predict(test_set)

PredictionOutput(predictions=array([[-0.02782909,  0.03326085],
       [-0.02784296,  0.03324939],
       [-0.02783708,  0.03324581],
       ...,
       [-0.02782897,  0.03324531],
       [-0.02784569,  0.03323086],
       [-0.0278399 ,  0.03325231]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 0, 0]), metrics={'eval_loss': 0.6932874917984009})


In this way, we can finetune the pre-trained BERT. Now that we have learned how to finetune the BERT for the text classification task, in the next section, let's see how to finetune the BERT model for the natural language inference task. 